In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px # plotly express
%matplotlib inline 
import pycountry
from geopy.geocoders import Nominatim

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
file_input=['/kaggle/input','../../../datasets/extracts/']
files={}
for dirname, _, filenames in os.walk(file_input[1]):
    for filename in filenames:
        files[filename]=os.path.join(dirname, filename)
        print(filename)

# Any results you write to the current directory are saved as output.


.DS_Store
countryLockdowndatesJHUMatch.csv
countryLockdowndates.csv
COVID19_open_line_list.csv
time_series_covid_19_confirmed_US.csv
time_series_covid_19_recovered.csv
time_series_covid_19_deaths_US.csv
covid_19_data.csv
COVID19_line_list_data.csv
time_series_covid_19_deaths.csv
time_series_covid_19_confirmed.csv


In [2]:
df=pd.read_csv(files['time_series_covid_19_confirmed.csv'])
# Pre-processing to remove negative data, if exists
df[df.columns[df.columns.str.contains('/20')]]=df[df.columns[df.columns.str.contains('/20')]].clip(lower=0)
country_col='Country/Region'
df.head()

Province/State Country/Region      Lat     Long  1/22/20  1/23/20  1/24/20  \
0            NaN    Afghanistan  33.0000  65.0000        0        0        0   
1            NaN        Albania  41.1533  20.1683        0        0        0   
2            NaN        Algeria  28.0339   1.6596        0        0        0   
3            NaN        Andorra  42.5063   1.5218        0        0        0   
4            NaN         Angola -11.2027  17.8739        0        0        0   

   1/25/20  1/26/20  1/27/20  ...  4/17/20  4/18/20  4/19/20  4/20/20  \
0        0        0        0  ...      906      933      996     1026   
1        0        0        0  ...      539      548      562      584   
2        0        0        0  ...     2418     2534     2629     2718   
3        0        0        0  ...      696      704      713      717   
4        0        0        0  ...       19       24       24       24   

   4/21/20  4/22/20  4/23/20  4/24/20  4/25/20  4/26/20  
0     1092     1176     1279     1351     1463     1531  
1      609      634      663      678      712      726  
2     2811     2910     3007     3127     3256     3382  
3      717      723      723      731      738      738  
4       24       25       25       25       25       26  

[5 rows x 100 columns]

In [9]:
locator = Nominatim(user_agent="myGeocoder")
def getIsoCodes(country_name,location):
    if pycountry.countries.get(name=country_name) is not None:
        return pycountry.countries.get(name=country_name).alpha_3
    elif pycountry.countries.get(alpha_2=country_name) is not None:
        return pycountry.countries.get(alpha_2=country_name).alpha_3
    else:
        location = locator.reverse(location)
        if 'address' in location.raw and'country_code' in location.raw['address'] and pycountry.countries.get(alpha_2=location.raw['address']['country_code'].upper()) is not None:
            return pycountry.countries.get(alpha_2=location.raw['address']['country_code'].upper()).alpha_3
        
        else:
            return ''

df['iso_codes']=df[[country_col,'Lat','Long']]\
        .apply(lambda record: getIsoCodes(record[country_col],', '.join(record[['Lat','Long']].astype(str).values)),axis=1)


In [29]:
confirmed_df=pd.melt(df[df.columns.difference(['Province/State','Lat','Long'])].groupby([country_col,'iso_codes']).sum().reset_index(),id_vars=[country_col,"iso_codes"], var_name="Date", value_name="Confirmed Cases")
# df[df.columns.difference(['Province/State','Lat','Long'])]

In [30]:
fig=px.choropleth(confirmed_df,
               locations='iso_codes',
               hover_name=country_col,
               animation_frame='Date',
               color='Confirmed Cases',
               color_continuous_scale=px.colors.sequential.Rainbow,
               projection="natural earth",
               title="Confirmed Cases over the world"
              )
fig.show()

In [31]:
confirmed_df['Date']=pd.to_datetime(confirmed_df['Date'])
confirmed_df['Date'].max().strftime('%-m/%-d/%y')

'4/26/20'

In [34]:
# top 10 countries
top_affected_countries=df.sort_values(confirmed_df['Date'].max().strftime('%-m/%-d/%y'),ascending=False)[country_col].iloc[:10].values

# df=df[df.columns.difference(['Lat','Long'])].groupby([country_col]).sum().reset_index()
# confirmed_df=pd.melt(df[df[country_col].isin(top_affected_countries)],id_vars=["Country/Region"], var_name="Date", value_name="Confirmed Cases")
# confirmed_df['Date']=pd.to_datetime(confirmed_df['Date'])
confirmed_df=confirmed_df[confirmed_df[country_col].isin(top_affected_countries)].sort_values('Date')
# confirmed_df['Country+Province']=confirmed_df["Country/Region"] + (confirmed_df["Province/State"].map(lambda x: '' if pd.isna(x) else ' ['+x+']'))

fig=px.line(confirmed_df, color=country_col, x='Date',y='Confirmed Cases')

fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [35]:
confirmed_pct_df=pd.concat([confirmed_df,confirmed_df.groupby([country_col])['Confirmed Cases'].pct_change().rename('Percentage Change')*100],axis=1)

fig=px.line(confirmed_pct_df, color=country_col, x='Date',y='Percentage Change')
fig.update_layout(yaxis = {'ticksuffix':'%'})
fig.update_xaxes(rangeslider_visible=True)
fig.show()

Empty DataFrame
Columns: [Country/Region, iso_codes, Date, Confirmed Cases]
Index: []